In [75]:
import pandas as pd
import re

<b>Step 1: Create a dataset for bots and non-bot tweets</b>

I take a dataset for the last 24 hours of tweets that I monitored. Then I simply run a for loop to check which account-ids can be found in the dataset more than 24 times and which are only in there once and save both lists. As the next part of the step, I use these lists to built a dataset of 1000 bot-tweets and 1000 non-bot-tweets.

In [76]:
modeldata=pd.read_csv("datasets/24to48.csv",  sep=',' , skipinitialspace = True, quotechar = "'")
modeldata.head()

created_at  \
0  Fri Aug 18 13:00:02 +0000 2017   
1  Fri Aug 18 13:00:06 +0000 2017   
2  Fri Aug 18 13:00:07 +0000 2017   
3  Fri Aug 18 13:00:08 +0000 2017   
4  Fri Aug 18 13:00:15 +0000 2017   

                                                text  \
0  "RT @sgt_flower: #Umfrage #Btw2017: Sind wir u...   
1  "RT @MarinellaSix: Angela #Merkel #CDU die mei...   
2  "@FOCUS_TopNews Es kann keinen Grund geben war...   
3  "meine Bürger in Deutschland – interessante Fo...   
4  "RT @BukowskisNephew: Das Wort Deutsche kommt ...   

                                             rt_text  \
0  "#Umfrage #Btw2017: Sind wir unter #Merkel noc...   
1  "Angela #Merkel #CDU die meistgehasste Frau #D...   
2                                                NaN   
3                                                NaN   
4  "Das Wort Deutsche kommt dieser Frau nicht meh...   

                                         quoted_text            tweet_id  \
0                                                NaN  898529925870870529   
1                                                NaN  898529940794224640   
2                                                NaN  898529947014352896   
3                                                NaN  898529949581275137   
4  "🆘‼️😠😡 #Merkel in Annaberg: We do not distingu...  898529978962370560   

  screen_name followers             user_id friends rt fav  
0       ukw11       175           183642388     244  0   0  
1    fhessen1       489          2721494964     474  0   0  
2      MrxFFM       189           319306373     320  0   0  
3   pa_szmann        17  873967112490029057      78  0   0  
4    mofu1995       208  787711969868603392     214  0   0

In [77]:
#sort the dataframe for the user_id
data=modeldata.sort_values(by='user_id')

IDbot=0
counter=0
botlist=[]

for cell in data["user_id"]:
    #if the counter is over 24, the id of the account is added to the botlist,
    #unless the id is already in there
    if counter>24:
        if IDbot not in botlist and IDbot != "user_id":
            botlist.append(IDbot)
        else:
            pass
    #if the current cell is not the same as the previously stored IDnor
    #and the counter is at 1, the ID is appended to the list of normal accounts
    if cell != IDbot:

            #the stored ID is changed to the cell-ID and counter resetted to 1
            IDbot=cell
            counter=1
    #otherwise the ID stays the same but the counter is increased by 1
    else:
        counter+=1

#this loop iterates through the csv and records which account-ids have tweeted only once
#and appends them to a list and is our "normal-list"
IDnor=0
#counter has to be set at the beginning to zero to avoid the first ID getting
#appended rightaway to the list of normal accounts
counter=0
normallist=[]
for cell in data["user_id"]:
#if the current cell is not the same as the previously stored IDnor
#and the counter is at 1, the ID is appended to the list of normal accounts
    if cell!=IDnor:
        if counter==1:
            normallist.append(IDnor)
            counter=0
        else:
            counter=0
    else:
        pass
    #this increases the counter
    counter+=1
    IDnor=cell
if (counter==1):
    normallist.append(IDnor)

As the next part of the step, I use these lists to flag each tweet as sent by a bot or not and then built a dataset of 1000 bot-tweets and 1000 non-bot-tweets.

In [79]:
reduced_data=pd.concat([data["text"], data["user_id"], data["tweet_id"]], axis=1)

botsampleid = []
normalsampleid = []
botsampletext=[]
normalsampletext=[]
bottweetid = []
normaltweetid = []

progcounter=0
#goes through the entire dataset and splits it up into two
for index, row in reduced_data.iterrows():
    progcounter+=1
    #if the id was in the botlist, the row goes to the bot-dataframe
    if(row['user_id'])in botlist:
        botsampleid.append(row['user_id'])
        botsampletext.append(row['text'])
        bottweetid.append(row['tweet_id'])

    #if the id was not in the botlist, the row goes to the normal-dataframe

    if(row['user_id'])not in botlist:
        normalsampleid.append(row['user_id'])
        normalsampletext.append(row['text'])
        normaltweetid.append(row['tweet_id'])


    #to see the progress, we use a counter
    if (progcounter%2000==0):
        print(progcounter)

2000
4000
6000
8000
10000
12000
14000
16000


In [ ]:
#we build a "bot" dataframe from these three lists and give it a fourth column
#with a default value of 1 for being a bot
botdf=pd.DataFrame()
botdf["user_id"]=botsampleid
botdf["text"]=botsampletext
botdf["tweet_id"]=  bottweetid
botdf["botornot"]=1

#we build a "normal" dataframe from these three lists and give it a fourth column
#with a default value of 1 for being a bot
normaldf=pd.DataFrame()
normaldf["user_id"]=normalsampleid
normaldf["text"]=normalsampletext
normaldf["tweet_id"]= normaltweetid
normaldf["botornot"]=0

#getting rid of the index column
normaldf.reset_index(drop=True, inplace=True)
botdf.reset_index(drop=True, inplace=True)
#removing all rows where the text is just 'text'
normaldf = normaldf[normaldf.text != 'text']
botdf = botdf[botdf.text != 'text']

#assembling from the collected dataframes two samples of size 1000
normaldf=(normaldf.sample(1000, replace=True))
botdf=(botdf.sample(1000, replace=True))


sample=pd.concat([normaldf,botdf], ignore_index=True)

<b>Step 2: Convert the tweets into a set of numbers</b>

In the next step, I converted these 2000 tweets into a list of numbers by counting properties like number of characters, number of unique words, number of exclamation marks etc. This is called a stylomertric approach.


In [85]:
data = sample

#the principle is as follows, first we create a list for each aspect that the script catches
# and then we puzzle together an array/dataframe from these lists
dotcountlist=[]
commacountlist=[]
exclamationcountlist=[]
wordcountlist=[]
uppercasecountlist=[]
lowercasecountlist=[]
charcountlist=[]
specialcharlist=[]
numbercountlist=[]
uniquewordlist=[]
qmarkcountlist=[]
apicescountlist=[]
quotescountlist=[]
openparlist=[]
closeparlist=[]
operatorlist=[]
hashtagcountlist=[]
dottycountlist=[]
linkcountlist=[]




a=0

for cell in (data['text']):

    a+=1
    dotcount=0
    commacount=0
    qmarkcount=0
    exclamationcount=0
    wordcount=0
    uppercasecount=0
    lowercasecount=0
    charcount=0
    specialchar=0
    numbercount=0
    cellwordlist=[]
    uniqueword=0
    qmarkcount=0
    apicescount=0
    quotescount=0
    openparcount=0
    closeparcount=0
    operatorcount=0
    hashtagcount=0
    dottycount=0
    linkcount=0

    #we now count how often certain characters appear in each text cell
    dotcount=cell.count(".")
    commacount=cell.count(",")
    exclamationcount=cell.count("!")
    qmarkcount=cell.count("?")
    apicescount=cell.count("'")
    quotescount=cell.count('"')
    openparcount=cell.count('(')
    closeparcount=cell.count(')')
    hashtagcount=cell.count('#')

    #now we check for each word in the cell
    for word in (cell.split()):
        word=str(word)

        #how many unique words we find
        if word in cellwordlist:
            continue
        else:
            cellwordlist.append(word)
            uniqueword+=1
        #how many "..." do we find
        if re.search('\.\.\.',word):
            dottycount+=1
        #how many links can we find
        if re.search('http',word):
            linkcount+=1

        #how many words are in each cell
        wordcount+=1

        #checks how many letters, that are not at the start of a word,
        #are written in uppercase like "WHAT ARE THEY DOING???"
        uppercaseword=word[1:]
        for c in uppercaseword:
            if c.istitle():
                uppercasecount+=1
                #print(word)

        #checks how many characters and numbers are in each word
        for char in word:
            charcount+=1
            #checks how many characters are used
            if re.match("[^a-zA-Z0-9]", char):
                specialchar+=1
            #checks how many numbers are used
            if re.match("[0-9]", char):
                numbercount+=1
            #checks how many characters are lowercase
            if char.islower():
                lowercasecount+=1
            #checks how many characters are operators
            if re.match("[\+\/\-\*\%\=<>&]", char):
                operatorcount+=1

    #appending the results of the various counters to different lists
    dotcountlist.append(dotcount)
    commacountlist.append(commacount)
    exclamationcountlist.append(exclamationcount)
    wordcountlist.append(wordcount)
    uppercasecountlist.append(uppercasecount)
    lowercasecountlist.append(lowercasecount)
    charcountlist.append(charcount)
    specialcharlist.append(specialchar)
    numbercountlist.append(numbercount)
    uniquewordlist.append(uniqueword)
    qmarkcountlist.append(qmarkcount)
    apicescountlist.append(apicescount)
    quotescountlist.append(quotescount)
    openparlist.append(openparcount)
    closeparlist.append(closeparcount)
    operatorlist.append(operatorcount)
    hashtagcountlist.append(hashtagcount)
    dottycountlist.append(dottycount)
    linkcountlist.append(linkcount)


#now we are building pandas series from these lists
row1=pd.DataFrame()
row1["Dotcount"]=dotcountlist
row2=pd.DataFrame()
row2["Commacount"]=commacountlist
row3=pd.DataFrame()
row3["Exclcount"]=exclamationcountlist
row4=pd.DataFrame()
row4["Wordcount"]=wordcountlist
row5=pd.DataFrame()
row5["Uppercasecount"]=uppercasecountlist
row6=pd.DataFrame()
row6["Lowercasecount"]=lowercasecountlist
row7=pd.DataFrame()
row7["Specialchar"]=specialcharlist
row8=pd.DataFrame()
row8["Numbercount"]=numbercountlist
row9=pd.DataFrame()
row9["Charcount"]=charcountlist
row10=pd.DataFrame()
row10["Uniquewords"]=uniquewordlist
row11=pd.DataFrame()
row11["Qmarkcount"]=qmarkcountlist
row11=pd.DataFrame()
row11["Apicescount"]=apicescountlist
row12=pd.DataFrame()
row12["Quotescount"]=quotescountlist
row13=pd.DataFrame()
row13["Openpar"]=openparlist
row14=pd.DataFrame()
row14["Closepar"]=closeparlist
row15=pd.DataFrame()
row15["Operatorcount"]=operatorlist
row16=pd.DataFrame()
row16["Hashtagcount"]=hashtagcountlist
row17=pd.DataFrame()
row17["Dottycount"]=dottycountlist
row18=pd.DataFrame()
row18["Linkcount"]=linkcountlist

#and we build a dataframe from these serieses
output=pd.concat([data, row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, row12, row13, row14, row15, row16, row17, row18],axis=1)
output.head()

user_id                                               text  \
0          2508227861  "RT @welchering: Wie ein Hackerangriff auf die...   
1  811544284344315909  "RT @eysvog3l: Aufgrund des großen Erfolges au...   
2           182353814  "#Merkel #Oberrabbiner von #Barcelona: „#Europ...   
3           131892594  "RT @ExSpiesser: Warum werden Menschen mit Beh...   
4            47435872  "RT @fabkienbaum: @CDU das Schönste an Eurem H...   

             tweet_id  botornot  Dotcount  Commacount  Exclcount  Wordcount  \
0  898799894592241664         0         0           0          0         13   
1  898877742422339584         0         0           0          0         16   
2  898611489455525888         0         1           0          0         10   
3  898583528841121793         0         1           1          0         17   
4  898868893183348737         0         1           0          0         15   

   Uppercasecount  Lowercasecount    ...      Charcount  Uniquewords  \
0               7              53    ...             77           13   
1               3             103    ...            118           16   
2              10              62    ...             90           10   
3               5              97    ...            123           17   
4              13              93    ...            128           15   

   Apicescount  Quotescount  Openpar  Closepar  Operatorcount  Hashtagcount  \
0            0            2        0         0              0             1   
1            0            2        0         0              0             1   
2            0            2        0         0              3             5   
3            0            2        0         0              1             2   
4            0            2        0         0              3             2   

   Dottycount  Linkcount  
0           0          0  
1           0          0  
2           0          1  
3           0          1  
4           0          1  

[5 rows x 22 columns]

In [86]:
#importing all machine learning libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix


<b>Step 3: Prepare the dataset for being read in by the algorithm</b>

Now, we have to take care of a couple of things. First, we want to make sure that the dependent variable (the one we want to predict), the "botornot"-criteria is stored separately from the rest of the dataset. 

In [87]:
#loading in the dataset
X = output

#defining the dependent variable as the botornot column in the dataset
y=X['botornot']

#removing user_id and bot_or_not column from the dataset with the independet variables
del X['botornot']
del X['user_id']
del X['tweet_id']

del X['text']

(X.head())

Dotcount  Commacount  Exclcount  Wordcount  Uppercasecount  Lowercasecount  \
0         0           0          0         13               7              53   
1         0           0          0         16               3             103   
2         1           0          0         10              10              62   
3         1           1          0         17               5              97   
4         1           0          0         15              13              93   

   Specialchar  Numbercount  Charcount  Uniquewords  Apicescount  Quotescount  \
0            9            6         77           13            0            2   
1            7            1        118           16            0            2   
2           17            1         90           10            0            2   
3           12            6        123           17            0            2   
4           19            1        128           15            0            2   

   Openpar  Closepar  Operatorcount  Hashtagcount  Dottycount  Linkcount  
0        0         0              0             1           0          0  
1        0         0              0             1           0          0  
2        0         0              3             5           0          1  
3        0         0              1             2           0          1  
4        0         0              3             2           0          1

Then we split the dataset into two parts. The training dataset and the test dataset on which we will check how accurate the algorithm is.


In [91]:
#splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

#defining rf as the RandomForestClassifier
clf=RandomForestClassifier()

<b>Step 4: Build the machine learning model, let it predict and check its accuracy</b>

With two lines of code, we train the algorithm on the training dataset and test its performance by running it against the test dataset. The performance stats that we look at are the accuracy, the mean absolute error and the confusion matrix. The documentation is here, but essentially, the more values are on the diagonal axis from the upper-left to the lower-right, the better is your algorithm.

In [106]:
#train the dataset with fitting
clf.fit(X_train, y_train)
#then run the prediction with the rest of the dataset
pred_test = clf.predict(X_test)

In [107]:
print ("Mean absolute error:",metrics.mean_absolute_error(y_test,pred_test))
print ("Accuracy:",metrics.accuracy_score(y_test,pred_test))
print (confusion_matrix(y_test, pred_test))

Mean absolute error: 0.235
Accuracy: 0.765
[[84 28]
 [19 69]]


<b>Step 5: Use the model to make predictions for a new, unknown dataset</b>

Finally, we run the random forest classifier against a new dataset to make predictions whether a tweet is a bot or not a bot-tweet.

In [94]:
datatopredict=pd.read_csv("datasets/Last24.csv",  sep=',' , skipinitialspace = True, quotechar = "'")
datatopredict.head()

created_at  \
0  Sat Aug 19 13:00:01 +0000 2017   
1  Sat Aug 19 13:00:02 +0000 2017   
2  Sat Aug 19 13:00:11 +0000 2017   
3  Sat Aug 19 13:00:15 +0000 2017   
4  Sat Aug 19 13:00:20 +0000 2017   

                                                text  \
0  "RT @connect17de: Los gehts beim #tweetup mit ...   
1  "RT @Harald70199: #Barcelona DAS IST EURE SCHU...   
2  "@EvaHoegl Hier für alle nochmal die Stellungn...   
3  "RT @b_riexinger: 16 Jahre #Merkel: @Die_Gruen...   
4  "RT @AfD_Bv_HH_Nord: #SKANDAL #SPD #Spitzenkan...   

                                             rt_text  \
0  "Los gehts beim #tweetup mit @peteraltmaier im...   
1  "#Barcelona DAS IST EURE SCHULD! DAS IST DIE S...   
2                                                NaN   
3  "16 Jahre #Merkel: @Die_Gruenen wollen es mögl...   
4  "#SKANDAL #SPD #Spitzenkandidatin Eva #Högl z....   

                                         quoted_text            tweet_id  \
0                                                NaN  898892309697142785   
1  "Graphic, 18+, watch on your own risk: →Scene ...  898892311580508161   
2                                                NaN  898892350260473857   
3  "Liebe @Die_Gruenen - das ist nicht Euer Ernst...  898892365926199297   
4                                                NaN  898892385781821444   

   screen_name followers             user_id friends rt fav  
0          CDU    206732            20429858    1190  0   0  
1    darktweep      3409  768755478671290368    3377  0   0  
2   striker632       152  757686932348493824     177  0   0  
3  FabioDeMasi      3168            14784765    1033  0   0  
4    romanus06       174          2635598825     161  0   0

In [95]:
data = datatopredict
data=pd.concat([data["text"], data["user_id"], data["tweet_id"]], axis=1)

#the principle is as follows, first we create a list for each aspect that the script catches
# and then we puzzle together an array/dataframe from these lists
dotcountlist=[]
commacountlist=[]
exclamationcountlist=[]
wordcountlist=[]
uppercasecountlist=[]
lowercasecountlist=[]
charcountlist=[]
specialcharlist=[]
numbercountlist=[]
uniquewordlist=[]
qmarkcountlist=[]
apicescountlist=[]
quotescountlist=[]
openparlist=[]
closeparlist=[]
operatorlist=[]
hashtagcountlist=[]
dottycountlist=[]
linkcountlist=[]




a=0

for cell in (data['text']):

    a+=1
    dotcount=0
    commacount=0
    qmarkcount=0
    exclamationcount=0
    wordcount=0
    uppercasecount=0
    lowercasecount=0
    charcount=0
    specialchar=0
    numbercount=0
    cellwordlist=[]
    uniqueword=0
    qmarkcount=0
    apicescount=0
    quotescount=0
    openparcount=0
    closeparcount=0
    operatorcount=0
    hashtagcount=0
    dottycount=0
    linkcount=0

    #we now count how often certain characters appear in each text cell
    dotcount=cell.count(".")
    commacount=cell.count(",")
    exclamationcount=cell.count("!")
    qmarkcount=cell.count("?")
    apicescount=cell.count("'")
    quotescount=cell.count('"')
    openparcount=cell.count('(')
    closeparcount=cell.count(')')
    hashtagcount=cell.count('#')

    #now we check for each word in the cell
    for word in (cell.split()):
        word=str(word)

        #how many unique words we find
        if word in cellwordlist:
            continue
        else:
            cellwordlist.append(word)
            uniqueword+=1
        #how many "..." do we find
        if re.search('\.\.\.',word):
            dottycount+=1
        #how many links can we find
        if re.search('http',word):
            linkcount+=1

        #how many words are in each cell
        wordcount+=1

        #checks how many letters, that are not at the start of a word,
        #are written in uppercase like "WHAT ARE THEY DOING???"
        uppercaseword=word[1:]
        for c in uppercaseword:
            if c.istitle():
                uppercasecount+=1
                #print(word)

        #checks how many characters and numbers are in each word
        for char in word:
            charcount+=1
            #checks how many characters are used
            if re.match("[^a-zA-Z0-9]", char):
                specialchar+=1
            #checks how many numbers are used
            if re.match("[0-9]", char):
                numbercount+=1
            #checks how many characters are lowercase
            if char.islower():
                lowercasecount+=1
            #checks how many characters are operators
            if re.match("[\+\/\-\*\%\=<>&]", char):
                operatorcount+=1

    #appending the results of the various counters to different lists
    dotcountlist.append(dotcount)
    commacountlist.append(commacount)
    exclamationcountlist.append(exclamationcount)
    wordcountlist.append(wordcount)
    uppercasecountlist.append(uppercasecount)
    lowercasecountlist.append(lowercasecount)
    charcountlist.append(charcount)
    specialcharlist.append(specialchar)
    numbercountlist.append(numbercount)
    uniquewordlist.append(uniqueword)
    qmarkcountlist.append(qmarkcount)
    apicescountlist.append(apicescount)
    quotescountlist.append(quotescount)
    openparlist.append(openparcount)
    closeparlist.append(closeparcount)
    operatorlist.append(operatorcount)
    hashtagcountlist.append(hashtagcount)
    dottycountlist.append(dottycount)
    linkcountlist.append(linkcount)


#now we are building pandas series from these lists
row1=pd.DataFrame()
row1["Dotcount"]=dotcountlist
row2=pd.DataFrame()
row2["Commacount"]=commacountlist
row3=pd.DataFrame()
row3["Exclcount"]=exclamationcountlist
row4=pd.DataFrame()
row4["Wordcount"]=wordcountlist
row5=pd.DataFrame()
row5["Uppercasecount"]=uppercasecountlist
row6=pd.DataFrame()
row6["Lowercasecount"]=lowercasecountlist
row7=pd.DataFrame()
row7["Specialchar"]=specialcharlist
row8=pd.DataFrame()
row8["Numbercount"]=numbercountlist
row9=pd.DataFrame()
row9["Charcount"]=charcountlist
row10=pd.DataFrame()
row10["Uniquewords"]=uniquewordlist
row11=pd.DataFrame()
row11["Qmarkcount"]=qmarkcountlist
row11=pd.DataFrame()
row11["Apicescount"]=apicescountlist
row12=pd.DataFrame()
row12["Quotescount"]=quotescountlist
row13=pd.DataFrame()
row13["Openpar"]=openparlist
row14=pd.DataFrame()
row14["Closepar"]=closeparlist
row15=pd.DataFrame()
row15["Operatorcount"]=operatorlist
row16=pd.DataFrame()
row16["Hashtagcount"]=hashtagcountlist
row17=pd.DataFrame()
row17["Dottycount"]=dottycountlist
row18=pd.DataFrame()
row18["Linkcount"]=linkcountlist

#and we build a dataframe from these serieses
datatopredict=pd.concat([data, row1, row2, row3, row4, row5, row6, row7, row8, row9, row10, row11, row12, row13, row14, row15, row16, row17, row18],axis=1)
datatopredict.head()

text             user_id  \
0  "RT @connect17de: Los gehts beim #tweetup mit ...            20429858   
1  "RT @Harald70199: #Barcelona DAS IST EURE SCHU...  768755478671290368   
2  "@EvaHoegl Hier für alle nochmal die Stellungn...  757686932348493824   
3  "RT @b_riexinger: 16 Jahre #Merkel: @Die_Gruen...            14784765   
4  "RT @AfD_Bv_HH_Nord: #SKANDAL #SPD #Spitzenkan...          2635598825   

             tweet_id  Dotcount  Commacount  Exclcount  Wordcount  \
0  898892309697142785         1           0          1         19   
1  898892311580508161         0           0          3         18   
2  898892350260473857         5           0          0         17   
3  898892365926199297         1           1          0         21   
4  898892385781821444         4           1          0         21   

   Uppercasecount  Lowercasecount  Specialchar    ...      Charcount  \
0               8              93           20    ...            127   
1              59              13           10    ...            100   
2              11              87           17    ...            118   
3               5              92           16    ...            116   
4              24              69           22    ...            117   

   Uniquewords  Apicescount  Quotescount  Openpar  Closepar  Operatorcount  \
0           19            0            2        0         0              4   
1           18            0            2        0         0              0   
2           17            0            2        0         0              3   
3           21            0            2        0         0              0   
4           21            0            2        0         0              0   

   Hashtagcount  Dottycount  Linkcount  
0             2           0          1  
1             2           0          0  
2             0           1          1  
3             1           0          0  
4             8           0          0  

[5 rows x 21 columns]

In [96]:
#loading in the dataset
X = datatopredict


del X['user_id']
del X['text']
del X['tweet_id']


(X.head())

Dotcount  Commacount  Exclcount  Wordcount  Uppercasecount  Lowercasecount  \
0         1           0          1         19               8              93   
1         0           0          3         18              59              13   
2         5           0          0         17              11              87   
3         1           1          0         21               5              92   
4         4           1          0         21              24              69   

   Specialchar  Numbercount  Charcount  Uniquewords  Apicescount  Quotescount  \
0           20            2        127           19            0            2   
1           10            5        100           18            0            2   
2           17            1        118           17            0            2   
3           16            2        116           21            0            2   
4           22            2        117           21            0            2   

   Openpar  Closepar  Operatorcount  Hashtagcount  Dottycount  Linkcount  
0        0         0              4             2           0          1  
1        0         0              0             2           0          0  
2        0         0              3             0           1          1  
3        0         0              0             1           0          0  
4        0         0              0             8           0          0

In [97]:
actualpred = clf.predict(datatopredict)

In [98]:
actualpredlist=actualpred.tolist()
actualpredlist

[0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,


In [99]:
#creating a series from the predicted bot or not count-list
rownew=pd.DataFrame()
rownew["MLbotornot"]=actualpredlist

In [101]:
finalresult=pd.concat([datatopredict, rownew], axis=1)
finalresult.head(10)

Dotcount  Commacount  Exclcount  Wordcount  Uppercasecount  Lowercasecount  \
0         1           0          1         19               8              93   
1         0           0          3         18              59              13   
2         5           0          0         17              11              87   
3         1           1          0         21               5              92   
4         4           1          0         21              24              69   
5         5           0          0         19               8              90   
6         1           0          0         19              15              77   
7         0           0          0         21               5             100   
8         0           0          2         13              21              88   
9         1           1          0         17               8             104   

   Specialchar  Numbercount  Charcount  Uniquewords  Apicescount  Quotescount  \
0           20            2        127           19            0            2   
1           10            5        100           18            0            2   
2           17            1        118           17            0            2   
3           16            2        116           21            0            2   
4           22            2        117           21            0            2   
5           23            0        125           19            0            2   
6           14            3        112           19            0            2   
7           11            3        123           21            0            2   
8           18            3        130           13            0            2   
9           12            0        126           17            0            2   

   Openpar  Closepar  Operatorcount  Hashtagcount  Dottycount  Linkcount  \
0        0         0              4             2           0          1   
1        0         0              0             2           0          0   
2        0         0              3             0           1          1   
3        0         0              0             1           0          0   
4        0         0              0             8           0          0   
5        1         1              3             3           0          1   
6        0         0              0             8           0          0   
7        0         0              0             1           0          0   
8        0         0              0             5           0          0   
9        0         0              3             0           0          1   

   MLbotornot  
0           0  
1           0  
2           0  
3           0  
4           1  
5           1  
6           1  
7           0  
8           1  
9           0

In [102]:
botstats=finalresult["MLbotornot"].value_counts()

In [105]:
print("number of normal tweets:",botstats[0])
print("number of bot tweets:",botstats[1])

number of normal tweets: 5925
number of bot tweets: 3774


In my master thesis, I actually used a combined approach out. Tweets were classified as bot-tweets if the account had tweeted more than 24 times per day or if the ML-algorithm determined as shown that it was a bot.


<b>Conclusion: What could be done better?</b>

In this code, I look at the stylometric features of tweets. However, it might have been more fruitful to look for distinctive features on the accounts’ profile pages rather than for each tweet. This is an approach that essentially SRF Data did on its Instagram influencer story that also used machine learning to determine how many followers of influencers were probably fake.
